# Diary for CRBM implementation



This notebook shows the parts from `crbm.py` with some details

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np 
import pandas as pd



##### read data from  `../Datasets/motion.mat`

More data from human motion captures can be found here:

http://people.csail.mit.edu/ehsu/work/sig05stf/

In [14]:
from scipy.io import loadmat  # this is the SciPy module that loads mat-files
data = loadmat('../Datasets/motion.mat')

In [17]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'skel', 'Motion'])

In [42]:
X1 = data["Motion"][0][0]
X2 = data["Motion"][0][1]
X3 = data["Motion"][0][2]

In [44]:
X1.shape, X2.shape, X2.shape

((1750, 108), (1040, 108), (1040, 108))

### CRBM class

In [63]:
a = 10
b=2
np.zeros([a,b])

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [150]:
class CRBM:
    def __init__(self, n_vis, n_hid, n_cond, seed=42, sigma=0.3, monitor_time=True):

        self.previous_xneg = None
        np.random.seed(seed)

        W = np.random.normal(0, sigma, [n_hid, n_vis])   # vis to hid
        A = np.random.normal(0, sigma, [n_vis, n_cond])  # cond to vis
        B = np.random.normal(0, sigma, [n_hid, n_cond])  # cond to hid

        v_bias = np.zeros([n_vis, 1]) 
        h_bias = np.zeros([n_hid, 1])

        dy_v_bias = np.zeros([n_vis, 1])
        dy_h_bias = np.zeros([n_hid, 1])

        self.W = np.array(W, dtype='float32')
        self.A = np.array(A, dtype='float32')
        self.B = np.array(B, dtype='float32')
        self.v_bias = v_bias
        self.h_bias = h_bias
        self.dy_v_bias = dy_v_bias
        self.dy_h_bias = dy_h_bias
        
        self.n_vis = n_vis
        self.n_hid = n_hid
        self.n_his = n_cond
        
        self.num_epochs_trained = 0
        self.lr = 0
        self.monitor_time = monitor_time       
    

In [151]:
crbm = CRBM(n_vis=108, n_hid=256, n_cond=20, seed=123, sigma = 0.3)

In [152]:
crbm.W.shape, crbm.A.shape, crbm.B.shape

((256, 108), (108, 20), (256, 20))

### Auxiliary functions

In [159]:
def split_vis(crbm: CRBM, vis: np.ndarray):
    
    # assert len(crbm.v)
    #n_features = crbm.n_vis
    vis_size = len(crbm.v_bias)
    x = vis[0:vis_size, :]
    cond = vis[(vis_size+1):(vis_size + crbm.A.shape[1]),:]
    return x, cond


def dynamic_biases_up(crbm: CRBM, cond: np.ndarray):
    crbm.dy_v_bias = np.dot(crbm.A, cond) + crbm.v_bias 
    crbm.dy_h_bias = np.dot(crbm.B, cond) + crbm.h_bias
    
    
def hid_means(crbm: CRBM, vis: np.ndarray):
    p = np.dot(crbm.W, vis) + crbm.dy_h_bias
    return sig(p)
    
    
def vis_means(crbm: CRBM, hid: np.ndarray):   
    p = np.dot(crbm.W.T, hid) + crbm.dy_v_bias
    return sig(p)


### Compute gradients

```
function gibbs(rbm::AbstractRBM, vis::Mat; n_times=1)
    v_pos = vis
    h_pos = sample_hiddens(rbm, v_pos)
    v_neg = sample_visibles(rbm, h_pos)
    h_neg = sample_hiddens(rbm, v_neg)
    for i=1:n_times-1
        v_neg = sample_visibles(rbm, h_neg)
        h_neg = sample_hiddens(rbm, v_neg)
    end
    return v_pos, h_pos, v_neg, h_neg
end
```

In [ ]:
def sample_hiddens(crbm, v_pos):
    
def sample_visibles(crbm, )

In [167]:
def CDK(crbm, vis, K=1):
    
    v_pos = vis
    h_pos = sample_hiddens(crbm, v_pos)
    v_neg = sample_visibles(crbm, h_pos)
    h_neg = sample_hiddens(rbm, v_neg)
    
    for i in range(K-1):
        v_neg = sample_visibles(rbm, h_neg)
        h_neg = sample_hiddens(rbm, v_neg)
    
    return v_pos, h_pos, v_neg, h_neg
    

In [160]:

def compute_gradient(crbm, X):
    """
    Computes an approximated gradient of the likelihod with respect to the parameters.
    """

    vis, cond = split_vis(crbm, X)
    
    v_pos, h_pos, v_neg, h_neg = CDK(crbm, vis)
    n_obs = vis.shape[1]
    
    # for a sigle observation:  dW = h * v^T - h_hat * v_hat^T
    dW = ( np.dot(h_pos, v_pos.T) - np.dot(h_neg, v_neg.T) ) * (1./n_obs)
    dA = ( np.dot(v_pos, cond.T)  - np.dot(v_neg, cond.T)  ) * (1./n_obs)
    dB = ( np.dot(h_pos, cond.T)  - np.dot(h_neg, cond.T)  ) * (1./n_obs) 
    
    dv_bias = np.mean(v_pos - v_neg, axis=1, keepdims=True)
    dh_bias = np.mean(h_pos - h_neg, axis=1, keepdims=True)
    
    return dW, dA, dB, dv_bias, dh_bias

In [161]:
X = X1[0:21,:]

In [162]:
X.shape, crbm.n_his

((21, 108), 20)

In [166]:
#compute_gradient(crbm, X)

### Apply momentum

### Train a batch of data

### Train whole data

### Make predictions with the model